In [10]:
import torch

# Verify if Cuda is available to use GPU
if torch.cuda.is_available():
    print("CUDA (GPU) está disponible en tu sistema.")
else:
    print("CUDA (GPU) no está disponible en tu sistema.")

CUDA (GPU) está disponible en tu sistema.


# AI by reinforcement learning 
---
This artificial intelligence will be able to learn to play and complete all levels of Geometry dash, using reinforcement learning. To complete this:
1. Connect python with the game
2. Define bounties and penalizations
3. Use Reinforcement learning like Q-learning, Deep Q-learning or Proximal Policy Optimization
4. Build a neuronal nerwork CNN? 
6. Training
7. Eval and tunning

## 1.Connect Python with the Game
---
This step will require:
1. Keyboard Emulation and Automation
- Description: Simulates keystrokes to control the game.
- Tools: Libraries such as pyautogui in Python can be used to simulate keystrokes.
- Challenges: Can be slow and not always accurate.
2. Use of Computer Vision Tools
- Description: Uses computer vision to "see" the state of the game and makes decisions based on that.
- Tools: Libraries such as OpenCV to process the visual output of the game.
- Challenges: Requires image processing and can be computationally intensive.
3. Creating a Custom Game Environment
- Description: Create a simplified version of Geometry Dash that you can fully control.
- Tools: Programming languages such as Python or JavaScript to create a basic version of the game.
- Challenges: Requires time and effort to replicate the game mechanics.

In [11]:
import pyautogui
import numpy as np
import time

def play_geometry_dash(model, capture_screen, process_image, get_reward, is_game_over, action_space, capture_rate=0.1):
    """
    Juega Geometry Dash con una IA.

    :param model: Modelo de IA para decidir acciones.
    :param capture_screen: Función para capturar pantalla.
    :param process_image: Función para procesar imágenes capturadas.
    :param get_reward: Función para calcular la recompensa.
    :param is_game_over: Función para determinar si el juego ha terminado.
    :param action_space: Espacio de acciones posibles.
    :param capture_rate: Tiempo entre capturas y acciones (en segundos).
    """

    state = capture_screen()  # Captura el estado inicial
    state = process_image(state)

    while True:
        action = model.predict_action(state)  # La IA elige una acción

        if action == action_space['click']:
            pyautogui.click()

        time.sleep(capture_rate)  # Espera antes de capturar el nuevo estado

        new_state = capture_screen()  # Captura el nuevo estado
        new_state = process_image(new_state)

        reward = get_reward(new_state)  # Calcula la recompensa
        done = is_game_over(new_state)  # Verifica si el juego terminó

        # Actualiza la memoria de repetición y entrena la IA
        model.update_replay_memory(state, action, reward, new_state, done)
        model.retrain()

        if done:
            break  # Fin del episodio

        state = new_state  # Actualiza el estado

    

Capture the game state

In [12]:
# (some auxiliary functions to cpt the game state)
# do actions
import cv2
import numpy as np
import pyautogui

# take a screenshot
def capture_screen(region=None, resize_dim=(128, 128)):
    # if region is None, will take the whole screen
    screen = pyautogui.screenshot(region=region)
    frame = np.array(screen)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, resize_dim)
    return frame

def perform_action(action):
    screen = capture_screen()
    processed = process_image(screen)
    bounty = get_bounty(screen)
    if action == 0:
        pyautogui.click()
    
    if action == 2:
        return process_image(processed), bounty, True

    return process_image(processed), bounty, False

def get_game_feedback():
    return get_bounty(), process_image(capture_screen())   

def process_image(image, stack_size=4):
    # Normalize
    processed_image = image / 255.0

    # Init stack
    if 'image_stack' not in globals():
        global image_stack
        image_stack = np.repeat(processed_image[..., np.newaxis], stack_size, axis=2)

    # Update stack images
    image_stack = np.append(image_stack[:, :, 1:], processed_image[..., np.newaxis], axis=2)

    return image_stack

In [13]:

def start_capturing_game(model):
    # Define the region to capt the game
    region = (0, 0, 1920, 1080)

    while True:

        screen = capture_screen(region)
        state = process_image(screen)
        # Predict the action to do
        action = model.predict_action(state)

        perform_action(action)

        # obtain reward and state
        reward, new_state = get_game_feedback() 

        # Update model
        model.update(state, action, reward, new_state)

        cv2.imshow("Game Capture", screen)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyAllWindows()



## 2 Define bounties

Now let's define the rewards, we will give 1 point for staying alive, and subtract when eliminated.

In [14]:
def detect_green_cube(screen):
    """
    Detects the presence of a green cube in an image (this will be our geometry skin).

    :param image_path: current screenshot
    :return: A tuple containing the percentage of green pixels and the mask image.
    """

    # Convert to HSV color space
    hsv_image = cv2.cvtColor(screen, cv2.COLOR_BGR2HSV)

    # Define the range of green color in HSV
    lower_green = np.array([40, 40, 40])
    upper_green = np.array([80, 255, 255])

    # Create a mask to detect green objects in the image
    green_mask = cv2.inRange(hsv_image, lower_green, upper_green)

    # Calculate the percentage of green pixels
    green_pixels = np.count_nonzero(green_mask)
    total_pixels = screen.shape[0] * screen.shape[1]
    green_percentage = (green_pixels / total_pixels) * 100
    
    # Return the percentage of green pixels and the mask for further analysis if needed
    return green_percentage, green_mask

def detect_if_win(screen):
    """
    Detects the presence of a green cube in an image (this will be our geometry skin).

    :param screen: current screenshot as a numpy array
    :return: A tuple containing the percentage of green pixels and the mask image.
    """

    # Convert to HSV color space
    hsv_image = cv2.cvtColor(screen, cv2.COLOR_BGR2HSV)

    # Define the range of yellow color in HSV
    lower_yellow = np.array([25, 100, 100])  # lower boundary of yellow hue
    upper_yellow = np.array([35, 255, 255])  # upper boundary of yellow hue


    # Create a mask to detect green objects in the image
    green_mask = cv2.inRange(hsv_image, lower_yellow, upper_yellow)

    # Calculate the percentage of green pixels
    green_pixels = np.count_nonzero(green_mask)
    total_pixels = screen.shape[0] * screen.shape[1]
    green_percentage = (green_pixels / total_pixels) * 100

    # Display the original image and the green mask
    #cv2.imshow("Original Image", screen)
    #cv2.imshow("Green Mask", green_mask)
   # cv2.waitKey(0)  # Waits for a key press to close the windows
   # cv2.destroyAllWindows()

    # Return the percentage of green pixels and the mask for further analysis if needed
    return green_percentage, green_mask

## Seems working properly detecting the block
def get_bounty(screen):
    green_percentage, green_mask = detect_green_cube(screen)

    if green_percentage > 0.35:
        return 1 
    else:
        yellow_percetage, yellow_mask = detect_if_win(screen)
        if yellow_percetage > 0.45 and yellow_percetage < 0.55:
            return 100
        return -10  


In [15]:
#time.sleep(5)
#detect_if_win(capture_screen())

## 3 Use Reinforcement learning
---


First of all we need a buffer of repetitions and some auxiliary functions

In [16]:
import random

class ReplayBuffer:
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = []
        self.position = 0

    def store(self, state, action, reward, next_state, done):
        if len(self.buffer) < self.capacity:
            self.buffer.append(None)
        self.buffer[self.position] = (state, action, reward, next_state, done)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)

    def __len__(self):
        return len(self.buffer)

def choose_action(state, model, epsilon):
    if np.random.rand() < epsilon:
        return np.random.choice(num_actions)
    else:
        q_values = model.predict(state)
        return np.argmax(q_values[0])

def update_network(model, target_model, minibatch, gamma):
    for state, action, reward, next_state, done in minibatch:
        target_q = reward
        if not done:
            target_q += gamma * np.amax(target_model.predict(next_state)[0])
        target_q_values = model.predict(state)
        target_q_values[0][action] = target_q

        model.fit(state, target_q_values, epochs=1, verbose=0)



create the model


In [17]:
from re import L
from tensorflow.keras import layers, models
from tensorflow.keras import regularizers
from tensorflow.keras.applications import MobileNetV2

base_model = MobileNetV2(input_shape=(128, 128, 3),
                         include_top=False)
def create_model():
  return models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.7),
    layers.Dense(102, activation='softmax')  # Oxford Flowers 102
])

Lets start to play

In [18]:
from tensorflow.keras.optimizers import Adam

# Parámetros
gamma = 0.99  # Discount 
epsilon = 1.0  # Epsilon to start epsilon-greedy politic
epsilon_min = 0.01  # min Epsilon
epsilon_decay = 0.995  # Factor de decaimiento de epsilon
learning_rate = 0.001  # Learning Rate
replay_buffer_capacity = 10000  # Max buffer capacity
minibatch_size = 32  # Tamaño del minibatch
num_episodes = 1000  # Número de episodios para entrenar
num_actions = 2  # Definir el número de acciones posibles
processed_image_shape = (128,128,3)  # Definir la forma de la imagen procesada

# Construir la red neuronal y el modelo objetivo
model = create_model()
target_model = create_model()
model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
target_model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')

time.sleep(5)  # Tiempo para preparar el juego

# Buffer de repetición
replay_buffer = ReplayBuffer(replay_buffer_capacity)

for episode in range(num_episodes):
    state = process_image(capture_screen())
    done = False
    total_reward = 0

    while not done:
        action = choose_action(state, model, epsilon)
        next_state, reward, done = perform_action(action)
        next_state = process_image(next_state)
        total_reward += reward

        replay_buffer.store(state, action, reward, next_state, done)
        state = next_state

        if len(replay_buffer) > minibatch_size:
            minibatch = replay_buffer.sample(minibatch_size)
            update_network(model, target_model, minibatch, gamma)

    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

    print(f'Episodio: {episode}, Recompensa total: {total_reward}, Epsilon: {epsilon}')


ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 4 dimension(s) and the array at index 1 has 5 dimension(s)